# Import data from google big query and store in local

## Imports and global declarations

In [8]:
#Only first time
#!pip3 install -r "../requirements.txt"

In [9]:
from sklearn import datasets
import sklearn
from sklearn import preprocessing
#from sklearn.externals import joblib
from sklearn import metrics
from sklearn import model_selection
import numpy as np
import pickle
import pandas as pd
from sklearn.linear_model import LinearRegression
import json
import os
os.environ.setdefault("GCLOUD_PROJECT", "turing-dev-337819")
from google.cloud import bigquery
client = bigquery.Client()

## Download raw data

In [10]:
sql = """
 -- Developers_Updated_Availability_Within_7_Days (binomial)
WITH
  __rawExperiment AS (
  SELECT
    jsonPayload.properties.feature AS feature,
    jsonPayload.properties.featurevalue AS featurevalue,
    jsonPayload.userid AS user_id,
    jsonPayload.properties.experimentKey AS experiment_id,
    jsonPayload.properties.experimentvariationid AS variation_id,
    timestamp AS timestamp,
  FROM
    `turing-230020.logging.EXPERIMENTS_*`
  WHERE
    timestamp BETWEEN '2022-06-28 00:00:00'
    AND '2022-07-29 18:07:00' ),
  __experiment AS (
    -- Viewed Experiment
  SELECT
    e.user_id AS user_id,
    CAST(e.variation_id AS string) AS variation,
    CAST(e.timestamp AS DATETIME) AS conversion_start,
    DATETIME_ADD(CAST(e.timestamp AS DATETIME),
      INTERVAL 168 HOUR) AS conversion_end
  FROM
    __rawExperiment e
  WHERE
    e.experiment_id = 'whatsapp_availability'
    AND CAST(e.timestamp AS DATETIME) >= DATETIME "2022-06-28 00:00:00"
    AND CAST(e.timestamp AS DATETIME) <= DATETIME "2022-07-29 18:07:00" ),
  __metric AS (
    -- Metric (Developers_Updated_Availability_Within_7_Days)
  SELECT
    user_id AS user_id,
    1 AS value,
    CAST(m.timestamp AS DATETIME) AS conversion_start,
    CAST(m.timestamp AS DATETIME) AS conversion_end
  FROM (
    WITH
      exp_table AS (
      SELECT
        CAST(exp.jsonPayload.userid AS INT) AS user_id,
        MIN(exp.timestamp) AS timestamp,
      FROM
        `turing-230020.logging.EXPERIMENTS_*` AS exp
      WHERE
        exp.jsonPayload.properties.experimentkey = 'whatsapp_availability'
        AND exp.timestamp >= '2022-06-24 00:00:00'
      GROUP BY
        exp.jsonPayload.userid ),
      table1 AS (
      SELECT
        payload.user_id,
        payload.action_user_id,
        payload.action_from,
        payload.answer AS answer,
        payload.notice_period,
        payload.developer_availability_type,
        payload.last_update
      FROM
        `turing-230020.mysql_cdc.turing_node_dv2_developer_availability`
      WHERE
        object = 'turing_node_dv2_developer_availability' ),
      final AS (
      SELECT
        t1.user_id,
        CAST(t1.user_id AS INT64) AS developer_id,
        answer,
        developer_availability_type,
        last_update,
        action_from
      FROM
        table1 t1
      WHERE
        last_update IS NOT NULL ),
      devs_lu AS (
      SELECT
        developer_id,
        MAX(last_update) AS max_last_update
      FROM
        final
      WHERE
        DATE(last_update) < DATE("2022-06-24")
      GROUP BY
        1 ),
      eligible_devs AS (
      SELECT
        *
      FROM
        devs_lu
      WHERE
        DATE(max_last_update) <= DATE_SUB(DATE("2022-06-24"), INTERVAL 15 DAY) ),
      distribution AS (
      SELECT
        *,
        CASE
          WHEN DATE_DIFF(DATE("2022-06-24"), DATE(max_last_update), DAY) >= 15 AND DATE_DIFF(DATE("2022-06-24"), DATE(max_last_update), DAY) <= 33 THEN '[15-33]'
        ELSE
        '33+'
      END
        AS days_before_launch
      FROM
        eligible_devs ),
      updates_after_launch AS (
      SELECT
        *
      FROM
        final
      LEFT JOIN
        exp_table AS et
      ON
        final.developer_id = et.user_id
      WHERE
        DATE(last_update) >= DATE(et.timestamp)
        AND developer_id IN (
        SELECT
          developer_id
        FROM
          eligible_devs ) ),
      min_update_after_launch AS (
      SELECT
        developer_id,
        answer,
        developer_availability_type,
        last_update,
        action_from
      FROM
        updates_after_launch t1
      WHERE
        last_update = (
        SELECT
          MIN(last_update)
        FROM
          updates_after_launch t2
        WHERE
          t1.developer_id = t2.developer_id ) -- and date_diff(date(last_update), date("2022-06-24"), DAY) <= 6
        ),
      cleaned_mul AS (
      SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY developer_id ORDER BY last_update ASC ) AS rank
      FROM
        min_update_after_launch ),
      cleaned_mul_first AS (
      SELECT
        *
      FROM
        cleaned_mul
      WHERE
        rank = 1 ),
      Final_data AS (
      SELECT
        ds.developer_id AS dev_id,
        CASE
          WHEN DATE_DIFF(DATE(cmf.last_update), DATE(et.timestamp), DAY) <= 6 THEN 1
        ELSE
        0
      END
        AS updated_7D,
        action_from,
        days_before_launch
      FROM
        distribution ds
      LEFT JOIN
        cleaned_mul_first cmf
      ON
        cmf.developer_id = ds.developer_id
      LEFT JOIN
        exp_table AS et
      ON
        et.user_id = ds.developer_id )
    SELECT
      CAST(exp.jsonPayload.userid AS INT) AS user_id,
      MIN(exp.timestamp) AS timestamp,
    FROM
      `turing-230020.logging.EXPERIMENTS_*` AS exp
    INNER JOIN
      Final_data AS fd
    ON
      exp.jsonPayload.userid = fd.dev_id
    WHERE
      fd.dev_id IS NOT NULL
      AND fd.updated_7D = 1
      AND exp.jsonPayload.properties.experimentkey = 'whatsapp_availability'
      AND exp.timestamp >= '2022-06-24 00:00:00'
    GROUP BY
      exp.jsonPayload.userid ) m
  WHERE
    CAST(m.timestamp AS DATETIME) >= DATETIME "2022-06-28 00:00:00"
    AND CAST(m.timestamp AS DATETIME) <= DATETIME "2022-08-05 18:07:00" ),
  __dimension AS (
    -- Dimension (wp_avaialbility_dev_valid_p2_pool)
  WITH
    exp_table AS (
    SELECT
      CAST(exp.jsonPayload.userid AS INT) AS user_id,
      MIN(exp.timestamp) AS timestamp,
    FROM
      `turing-230020.logging.EXPERIMENTS_*` AS exp
    WHERE
      exp.jsonPayload.properties.experimentkey = 'whatsapp_availability'
      AND exp.timestamp >= '2022-06-24 00:00:00'
    GROUP BY
      exp.jsonPayload.userid ),
    tbl1 AS (
    SELECT
      dch.developer_id,
      dch.new_value AS staus_before_message,
      dch.time_changed,
      et.timestamp,
      CASE
        WHEN p2.phase2_entry_date <= et.timestamp THEN "P2"
      ELSE
      "P1"
    END
      AS phase_flag,
      CASE
        WHEN dch.new_value IN ( 'passed ti', 'available supply', 'standardized resume', 'ready 2.0' ) THEN "P2_valid"
        WHEN p2.phase2_entry_date IS NULL THEN "P1"
      ELSE
      "other"
    END
      AS valid_p2,
      ROW_NUMBER() OVER(PARTITION BY dch.developer_id ORDER BY time_changed DESC ) AS rn
    FROM
      `turing-230020.devdb_mirror.developer_changes_history` AS dch
    INNER JOIN
      exp_table AS et
    ON
      dch.developer_id = et.user_id
    LEFT JOIN
      `curated.phase2_dev_level_data` AS p2
    ON
      dch.developer_id = p2.dev_id
    WHERE
      TABLE = 'developer_detail'
      AND COLUMN = 'status'
      AND dch.time_changed <= et.timestamp )
  SELECT
    developer_id AS user_id,
    valid_p2 AS value
  FROM
    tbl1
  WHERE
    rn = 1
  ORDER BY
    developer_id ASC ),
  __distinctUsers AS (
    -- One row per user/dimension
  SELECT
    e.user_id,
    d.value AS dimension,
    (
      CASE
        WHEN COUNT(DISTINCT e.variation) > 1 THEN '__multiple__'
      ELSE
      MAX(e.variation)
    END
      ) AS variation,
    MIN(e.conversion_start) AS conversion_start,
    MIN(e.conversion_end) AS conversion_end
  FROM
    __experiment e
  JOIN
    __dimension d
  ON
    (d.user_id = e.user_id)
  GROUP BY
    dimension,
    e.user_id ),
  __userMetricId AS (
    -- Add in the aggregate metric value for each user
  SELECT DISTINCT
    d.variation,
    d.dimension,
    d.user_id
  FROM
    __distinctUsers d
  JOIN
    __metric m
  ON
    (m.user_id = d.user_id)
  WHERE
    m.conversion_start >= d.conversion_start
    AND m.conversion_start <= d.conversion_end
  GROUP BY
    variation,
    dimension,
    d.user_id ),
  __userMetric AS (
    -- Add in the aggregate metric value for each user
  SELECT
    d.variation,
    d.dimension,
    1 AS value
  FROM
    __distinctUsers d
  JOIN
    __metric m
  ON
    (m.user_id = d.user_id)
  WHERE
    m.conversion_start >= d.conversion_start
    AND m.conversion_start <= d.conversion_end
  GROUP BY
    variation,
    dimension,
    d.user_id ),
  __overallUsers AS (
    -- Number of users in each variation
  SELECT
    variation,
    dimension,
    COUNT(*) AS users
  FROM
    __distinctUsers
  GROUP BY
    variation,
    dimension ),
  __stats AS (
    -- Sum all user metrics together to get a total per variation/dimension
  SELECT
    variation,
    dimension,
    COUNT(*) AS count,
    AVG(value) AS mean,
    STDDEV(value) AS stddev
  FROM
    __userMetric
  GROUP BY
    variation,
    dimension ),

  conversion AS (
  SELECT
    cluster_label,
    COUNT(dev_id) AS conversion_cnt,
    ROUND(COUNT(dev_id)/MAX(total_cnt),2) AS conversion_percentage
  FROM (
    SELECT
      cluster_label,
      dev_id,
      COUNT(dev_id) OVER() AS total_cnt
    FROM
      `turing-dev-337819.pdsa.PDAS_P2_cluster` AS c
    INNER JOIN
      __userMetricId AS d
    ON
      d.user_id = c.dev_id
    WHERE
      d.variation = "0"
      AND d.dimension = "P2_valid") AS a
  GROUP BY
    1
  ORDER BY
    1),
  exposed AS(
  SELECT
    cluster_label,
    COUNT(dev_id) AS exposed_cnt,
    ROUND(COUNT(dev_id)/MAX(total_cnt),2) AS exposed_percentage
  FROM (
    SELECT
      cluster_label,
      dev_id,
      COUNT(dev_id) OVER() AS total_cnt
    FROM
      `turing-dev-337819.pdsa.PDAS_P2_cluster` AS c
    INNER JOIN
      __distinctUsers AS d
    ON
      d.user_id = c.dev_id
    WHERE
      d.variation = "0"
      AND d.dimension = "P2_valid") AS a
  GROUP BY
    1
  ORDER BY
    1),
  population AS (
  SELECT
    cluster_label,
    COUNT(dev_id) AS population_cnt,
    ROUND(COUNT(dev_id)/MAX(total_cnt),2) AS population_cnt
  FROM (
    SELECT
      cluster_label,
      dev_id,
      COUNT(dev_id) OVER() AS total_cnt
    FROM
      `turing-dev-337819.pdsa.PDAS_P2_cluster` AS c) AS a
  GROUP BY
    1
  ORDER BY
    1)

  -- SELECT
  --   s.variation,
  --   s.dimension,
  --   s.count,
  --   s.mean,
  --   s.stddev,
  --   u.users
  -- FROM
  --   __stats s
  --   JOIN __overallUsers u ON (
  --       s.variation = u.variation
  --       AND s.dimension = u.dimension
  --     )


SELECT *, ROUND((conversion_cnt/exposed_cnt) * 100,2) as conversion_rate  FROM population
LEFT JOIN exposed
USING(cluster_label)
LEFT JOIN conversion
USING(cluster_label)
ORDER BY conversion_rate
"""
exp_results_test = client.query(sql).to_dataframe()

In [13]:
sql = """
 -- Developers_Updated_Availability_Within_7_Days (binomial)
WITH
  __rawExperiment AS (
  SELECT
    jsonPayload.properties.feature AS feature,
    jsonPayload.properties.featurevalue AS featurevalue,
    jsonPayload.userid AS user_id,
    jsonPayload.properties.experimentKey AS experiment_id,
    jsonPayload.properties.experimentvariationid AS variation_id,
    timestamp AS timestamp,
  FROM
    `turing-230020.logging.EXPERIMENTS_*`
  WHERE
    timestamp BETWEEN '2022-06-28 00:00:00'
    AND '2022-07-29 18:07:00' ),
  __experiment AS (
    -- Viewed Experiment
  SELECT
    e.user_id AS user_id,
    CAST(e.variation_id AS string) AS variation,
    CAST(e.timestamp AS DATETIME) AS conversion_start,
    DATETIME_ADD(CAST(e.timestamp AS DATETIME),
      INTERVAL 168 HOUR) AS conversion_end
  FROM
    __rawExperiment e
  WHERE
    e.experiment_id = 'whatsapp_availability'
    AND CAST(e.timestamp AS DATETIME) >= DATETIME "2022-06-28 00:00:00"
    AND CAST(e.timestamp AS DATETIME) <= DATETIME "2022-07-29 18:07:00" ),
  __metric AS (
    -- Metric (Developers_Updated_Availability_Within_7_Days)
  SELECT
    user_id AS user_id,
    1 AS value,
    CAST(m.timestamp AS DATETIME) AS conversion_start,
    CAST(m.timestamp AS DATETIME) AS conversion_end
  FROM (
    WITH
      exp_table AS (
      SELECT
        CAST(exp.jsonPayload.userid AS INT) AS user_id,
        MIN(exp.timestamp) AS timestamp,
      FROM
        `turing-230020.logging.EXPERIMENTS_*` AS exp
      WHERE
        exp.jsonPayload.properties.experimentkey = 'whatsapp_availability'
        AND exp.timestamp >= '2022-06-24 00:00:00'
      GROUP BY
        exp.jsonPayload.userid ),
      table1 AS (
      SELECT
        payload.user_id,
        payload.action_user_id,
        payload.action_from,
        payload.answer AS answer,
        payload.notice_period,
        payload.developer_availability_type,
        payload.last_update
      FROM
        `turing-230020.mysql_cdc.turing_node_dv2_developer_availability`
      WHERE
        object = 'turing_node_dv2_developer_availability' ),
      final AS (
      SELECT
        t1.user_id,
        CAST(t1.user_id AS INT64) AS developer_id,
        answer,
        developer_availability_type,
        last_update,
        action_from
      FROM
        table1 t1
      WHERE
        last_update IS NOT NULL ),
      devs_lu AS (
      SELECT
        developer_id,
        MAX(last_update) AS max_last_update
      FROM
        final
      WHERE
        DATE(last_update) < DATE("2022-06-24")
      GROUP BY
        1 ),
      eligible_devs AS (
      SELECT
        *
      FROM
        devs_lu
      WHERE
        DATE(max_last_update) <= DATE_SUB(DATE("2022-06-24"), INTERVAL 15 DAY) ),
      distribution AS (
      SELECT
        *,
        CASE
          WHEN DATE_DIFF(DATE("2022-06-24"), DATE(max_last_update), DAY) >= 15 AND DATE_DIFF(DATE("2022-06-24"), DATE(max_last_update), DAY) <= 33 THEN '[15-33]'
        ELSE
        '33+'
      END
        AS days_before_launch
      FROM
        eligible_devs ),
      updates_after_launch AS (
      SELECT
        *
      FROM
        final
      LEFT JOIN
        exp_table AS et
      ON
        final.developer_id = et.user_id
      WHERE
        DATE(last_update) >= DATE(et.timestamp)
        AND developer_id IN (
        SELECT
          developer_id
        FROM
          eligible_devs ) ),
      min_update_after_launch AS (
      SELECT
        developer_id,
        answer,
        developer_availability_type,
        last_update,
        action_from
      FROM
        updates_after_launch t1
      WHERE
        last_update = (
        SELECT
          MIN(last_update)
        FROM
          updates_after_launch t2
        WHERE
          t1.developer_id = t2.developer_id ) -- and date_diff(date(last_update), date("2022-06-24"), DAY) <= 6
        ),
      cleaned_mul AS (
      SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY developer_id ORDER BY last_update ASC ) AS rank
      FROM
        min_update_after_launch ),
      cleaned_mul_first AS (
      SELECT
        *
      FROM
        cleaned_mul
      WHERE
        rank = 1 ),
      Final_data AS (
      SELECT
        ds.developer_id AS dev_id,
        CASE
          WHEN DATE_DIFF(DATE(cmf.last_update), DATE(et.timestamp), DAY) <= 6 THEN 1
        ELSE
        0
      END
        AS updated_7D,
        action_from,
        days_before_launch
      FROM
        distribution ds
      LEFT JOIN
        cleaned_mul_first cmf
      ON
        cmf.developer_id = ds.developer_id
      LEFT JOIN
        exp_table AS et
      ON
        et.user_id = ds.developer_id )
    SELECT
      CAST(exp.jsonPayload.userid AS INT) AS user_id,
      MIN(exp.timestamp) AS timestamp,
    FROM
      `turing-230020.logging.EXPERIMENTS_*` AS exp
    INNER JOIN
      Final_data AS fd
    ON
      exp.jsonPayload.userid = fd.dev_id
    WHERE
      fd.dev_id IS NOT NULL
      AND fd.updated_7D = 1
      AND exp.jsonPayload.properties.experimentkey = 'whatsapp_availability'
      AND exp.timestamp >= '2022-06-24 00:00:00'
    GROUP BY
      exp.jsonPayload.userid ) m
  WHERE
    CAST(m.timestamp AS DATETIME) >= DATETIME "2022-06-28 00:00:00"
    AND CAST(m.timestamp AS DATETIME) <= DATETIME "2022-08-05 18:07:00" ),
  __dimension AS (
    -- Dimension (wp_avaialbility_dev_valid_p2_pool)
  WITH
    exp_table AS (
    SELECT
      CAST(exp.jsonPayload.userid AS INT) AS user_id,
      MIN(exp.timestamp) AS timestamp,
    FROM
      `turing-230020.logging.EXPERIMENTS_*` AS exp
    WHERE
      exp.jsonPayload.properties.experimentkey = 'whatsapp_availability'
      AND exp.timestamp >= '2022-06-24 00:00:00'
    GROUP BY
      exp.jsonPayload.userid ),
    tbl1 AS (
    SELECT
      dch.developer_id,
      dch.new_value AS staus_before_message,
      dch.time_changed,
      et.timestamp,
      CASE
        WHEN p2.phase2_entry_date <= et.timestamp THEN "P2"
      ELSE
      "P1"
    END
      AS phase_flag,
      CASE
        WHEN dch.new_value IN ( 'passed ti', 'available supply', 'standardized resume', 'ready 2.0' ) THEN "P2_valid"
        WHEN p2.phase2_entry_date IS NULL THEN "P1"
      ELSE
      "other"
    END
      AS valid_p2,
      ROW_NUMBER() OVER(PARTITION BY dch.developer_id ORDER BY time_changed DESC ) AS rn
    FROM
      `turing-230020.devdb_mirror.developer_changes_history` AS dch
    INNER JOIN
      exp_table AS et
    ON
      dch.developer_id = et.user_id
    LEFT JOIN
      `curated.phase2_dev_level_data` AS p2
    ON
      dch.developer_id = p2.dev_id
    WHERE
      TABLE = 'developer_detail'
      AND COLUMN = 'status'
      AND dch.time_changed <= et.timestamp )
  SELECT
    developer_id AS user_id,
    valid_p2 AS value
  FROM
    tbl1
  WHERE
    rn = 1
  ORDER BY
    developer_id ASC ),
  __distinctUsers AS (
    -- One row per user/dimension
  SELECT
    e.user_id,
    d.value AS dimension,
    (
      CASE
        WHEN COUNT(DISTINCT e.variation) > 1 THEN '__multiple__'
      ELSE
      MAX(e.variation)
    END
      ) AS variation,
    MIN(e.conversion_start) AS conversion_start,
    MIN(e.conversion_end) AS conversion_end
  FROM
    __experiment e
  JOIN
    __dimension d
  ON
    (d.user_id = e.user_id)
  GROUP BY
    dimension,
    e.user_id ),
  __userMetricId AS (
    -- Add in the aggregate metric value for each user
  SELECT DISTINCT
    d.variation,
    d.dimension,
    d.user_id
  FROM
    __distinctUsers d
  JOIN
    __metric m
  ON
    (m.user_id = d.user_id)
  WHERE
    m.conversion_start >= d.conversion_start
    AND m.conversion_start <= d.conversion_end
  GROUP BY
    variation,
    dimension,
    d.user_id ),
  __userMetric AS (
    -- Add in the aggregate metric value for each user
  SELECT
    d.variation,
    d.dimension,
    1 AS value
  FROM
    __distinctUsers d
  JOIN
    __metric m
  ON
    (m.user_id = d.user_id)
  WHERE
    m.conversion_start >= d.conversion_start
    AND m.conversion_start <= d.conversion_end
  GROUP BY
    variation,
    dimension,
    d.user_id ),
  __overallUsers AS (
    -- Number of users in each variation
  SELECT
    variation,
    dimension,
    COUNT(*) AS users
  FROM
    __distinctUsers
  GROUP BY
    variation,
    dimension ),
  __stats AS (
    -- Sum all user metrics together to get a total per variation/dimension
  SELECT
    variation,
    dimension,
    COUNT(*) AS count,
    AVG(value) AS mean,
    STDDEV(value) AS stddev
  FROM
    __userMetric
  GROUP BY
    variation,
    dimension ),

  conversion AS (
  SELECT
    cluster_label,
    COUNT(dev_id) AS conversion_cnt,
    ROUND(COUNT(dev_id)/MAX(total_cnt),2) AS conversion_percentage
  FROM (
    SELECT
      cluster_label,
      dev_id,
      COUNT(dev_id) OVER() AS total_cnt
    FROM
      `turing-dev-337819.pdsa.PDAS_P2_cluster` AS c
    INNER JOIN
      __userMetricId AS d
    ON
      d.user_id = c.dev_id
    WHERE
      d.variation = "1"
      AND d.dimension = "P2_valid") AS a
  GROUP BY
    1
  ORDER BY
    1),
  exposed AS(
  SELECT
    cluster_label,
    COUNT(dev_id) AS exposed_cnt,
    ROUND(COUNT(dev_id)/MAX(total_cnt),2) AS exposed_percentage
  FROM (
    SELECT
      cluster_label,
      dev_id,
      COUNT(dev_id) OVER() AS total_cnt
    FROM
      `turing-dev-337819.pdsa.PDAS_P2_cluster` AS c
    INNER JOIN
      __distinctUsers AS d
    ON
      d.user_id = c.dev_id
    WHERE
      d.variation = "1"
      AND d.dimension = "P2_valid") AS a
  GROUP BY
    1
  ORDER BY
    1),
  population AS (
  SELECT
    cluster_label,
    COUNT(dev_id) AS population_cnt,
    ROUND(COUNT(dev_id)/MAX(total_cnt),2) AS population_cnt
  FROM (
    SELECT
      cluster_label,
      dev_id,
      COUNT(dev_id) OVER() AS total_cnt
    FROM
      `turing-dev-337819.pdsa.PDAS_P2_cluster` AS c) AS a
  GROUP BY
    1
  ORDER BY
    1)

  -- SELECT
  --   s.variation,
  --   s.dimension,
  --   s.count,
  --   s.mean,
  --   s.stddev,
  --   u.users
  -- FROM
  --   __stats s
  --   JOIN __overallUsers u ON (
  --       s.variation = u.variation
  --       AND s.dimension = u.dimension
  --     )


SELECT *, ROUND((conversion_cnt/exposed_cnt) * 100,2) as conversion_rate  FROM population
LEFT JOIN exposed
USING(cluster_label)
LEFT JOIN conversion
USING(cluster_label)
ORDER BY conversion_rate
"""
exp_results_control = client.query(sql).to_dataframe()

## Data preprocessing

In [14]:
exp_results_test

,cluster_label,population_cnt,population_cnt_1,exposed_cnt,exposed_percentage,conversion_cnt,conversion_percentage,conversion_rate
0,Elite,1999,0.04,87,0.03,12,0.02,13.79
1,High Quality,9847,0.21,606,0.18,126,0.16,20.79
2,Average,12538,0.27,1143,0.33,244,0.32,21.35
3,Low Quality,11446,0.24,865,0.25,203,0.27,23.47
4,Low Experience,10931,0.23,747,0.22,180,0.24,24.10


In [15]:
exp_results_control

,cluster_label,population_cnt,population_cnt_1,exposed_cnt,exposed_percentage,conversion_cnt,conversion_percentage,conversion_rate
0,High Quality,9847,0.21,510,0.16,41,0.14,8.04
1,Low Quality,11446,0.24,884,0.27,79,0.27,8.94
2,Average,12538,0.27,1060,0.33,97,0.33,9.15
3,Low Experience,10931,0.23,702,0.22,68,0.23,9.69
4,Elite,1999,0.04,85,0.03,9,0.03,10.59


## Store and version raw data

In [16]:
exp_results_test.to_csv('../data/raw/exp_results_test.csv', index_label='Index')
exp_results_control.to_csv('../data/raw/exp_results_control.csv', index_label='Index')